<a href="https://cognitiveclass.ai/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkRP0321ENSkillsNetwork25371262-2022-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0101EN-Coursera/v2/M1_R_Basics/images/IDSNlogo.png" width="200" align="center">
</a>


<h1>Data Wrangling with Regular Expressions</h1>

Estimated time needed: **40** minutes


## Lab Overview:

In the previous data collection labs, you collected some raw datasets from several different sources. In this lab, you need to perform data wrangling tasks in order to improve data quality.


You will again use regular expressions, along with the `stringr` package (part of `tidyverse`), to clean up the bike-sharing systems data that you previously web scraped from the wiki page:

[https://en.wikipedia.org/wiki/List_of_bicycle-sharing_systems](https://en.wikipedia.org/wiki/List_of_bicycle-sharing_systems?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkRP0321ENSkillsNetwork25371262-2022-01-01)

<a href="https://cognitiveclass.ai/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkRP0321ENSkillsNetwork25371262-2022-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/module_1/images/l2-list-bike-sharing-systems.png" width="800" align="center">
</a>


One typical challenge of web scraping is that data extracted from HTML pages may contain unnecessary or inconsistently fomatted information.\
For example:

*   Textual annotations in numeric fields: `1000 (Updated with 1050)`
*   Attached reference links: `Bike sharing system [123]`
*   Inconsistent data formats: `Yes` and `Y` for the logical value `TRUE` or `2021-04-09` and `Apr 09, 2021` for the same date
*   HTML style tags: `<span style="color:blue">Bike sharing system</span>`
*   Special characters: `&nbsp` for a white space

Many more such examples of noise may be encountered in real-world scraped data and most of such text related noises could be handled by regular expressions.


To summarize, you will be using `stringr` (part of `tidyverse`) and regular expressions to perform the following data wrangling tasks:

*   TASK: Standardize column names for all collected datasets
*   TASK: Remove undesired reference links from the scraped bike-sharing systems dataset
*   TASK: Extract only the numeric value from undesired text annotations


Let's begin by importing the libraries you will use for these data wrangling tasks.


In [9]:
# Check whether you need to install the `tidyverse` library
require("tidyverse")
library(tidyverse)
library(stringr)
library(dplyr)

## TASK: Standardize column names for all collected datasets


In the previous data collection labs, you collected four datasets in csv format:

*   `raw_bike_sharing_systems.csv`:  A list of active bike-sharing systems across the world
*   `raw_cities_weather_forecast.csv`: 5-day weather forecasts for a list of cities, from OpenWeather API
*   `raw_worldcities.csv`: A list of major cities' info (such as name, latitude and longitude) across the world
*   `raw_seoul_bike_sharing.csv`: Weather information (Temperature, Humidity, Windspeed, Visibility, Dewpoint, Solar radiation, Snowfall, Rainfall), the number of bikes rented per hour, and date information, from Seoul bike-sharing systems


*Optional:* If you had some difficulties finishing the data collection labs, you may download the datasets directly from the following URLs:


In [10]:
# Download raw_bike_sharing_systems.csv
url <- "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/raw_bike_sharing_systems.csv"
download.file(url, destfile = "raw_bike_sharing_systems.csv")

# Download raw_cities_weather_forecast.csv
url <- "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/raw_cities_weather_forecast.csv"
download.file(url, destfile = "raw_cities_weather_forecast.csv")

# Download raw_worldcities.csv
url <- "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/raw_worldcities.csv"
download.file(url, destfile = "raw_worldcities.csv")

# Download raw_seoul_bike_sharing.csv
url <- "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/raw_seoul_bike_sharing.csv"
download.file(url, destfile = "raw_seoul_bike_sharing.csv")

To improve dataset readbility by both human and computer systems, we first need to standardize the column names of the datasets above using the following naming convention:

*   Column names need to be UPPERCASE
*   The word separator needs to be an underscore, such as in `COLUMN_NAME`


In [11]:
bikes<-read.csv("raw_bike_sharing_systems.csv")
seoul_bike<-read.csv("raw_seoul_bike_sharing.csv")
cities_weather<-read.csv("raw_cities_weather_forecast.csv")
worldcities_weather<-read.csv("raw_worldcities.csv")
head(cities_weather)
head(seoul_bike)
head(bikes)
head(worldcities_weather)

,city,weather,visibility,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,season,forecast_datetime
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<int>,<chr>,<chr>
1,Seoul,Clear,10000,12.32,10.91,12.32,1015,50,2.18,248,Spring,2021-04-16 12:00:00
2,Seoul,Clear,10000,11.48,9.81,11.48,1016,48,1.25,142,Spring,2021-04-16 15:00:00
3,Seoul,Clouds,10000,9.99,8.82,9.99,1015,46,0.94,130,Spring,2021-04-16 18:00:00
4,Seoul,Clouds,10000,7.87,7.87,7.87,1014,46,0.83,31,Spring,2021-04-16 21:00:00
5,Seoul,Clouds,10000,10.09,10.09,10.09,1014,37,1.96,309,Spring,2021-04-17 00:00:00
6,Seoul,Rain,10000,9.74,9.74,9.74,1014,48,3.24,267,Spring,2021-04-17 03:00:00


,Date,RENTED_BIKE_COUNT,Hour,TEMPERATURE,HUMIDITY,WIND_SPEED,Visibility,DEW_POINT_TEMPERATURE,SOLAR_RADIATION,RAINFALL,Snowfall,SEASONS,HOLIDAY,FUNCTIONING_DAY
,<chr>,<int>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
1,01/12/2017,254,0,-5.2,37,2.2,2000,-17.6,0,0,0,Winter,No Holiday,Yes
2,01/12/2017,204,1,-5.5,38,0.8,2000,-17.6,0,0,0,Winter,No Holiday,Yes
3,01/12/2017,173,2,-6.0,39,1.0,2000,-17.7,0,0,0,Winter,No Holiday,Yes
4,01/12/2017,107,3,-6.2,40,0.9,2000,-17.6,0,0,0,Winter,No Holiday,Yes
5,01/12/2017,78,4,-6.0,36,2.3,2000,-18.6,0,0,0,Winter,No Holiday,Yes
6,01/12/2017,100,5,-6.4,37,1.5,2000,-18.7,0,0,0,Winter,No Holiday,Yes


,COUNTRY,City,Name,SYSTEM,OPERATOR,LAUNCHED,DISCONTINUED,STATIONS,BICYCLES,DAILY_RIDERSHIP
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Albania,Tirana,Ecovolis,NA,NA,March 2011,NA,8,200,NA
2,Argentina,Mendoza,Metrobici,NA,NA,2014,NA,2,40,NA
3,Argentina,"San Lorenzo, Santa Fe",Biciudad,Biciudad,NA,27 November 2016,NA,8,80,NA
4,Argentina,Buenos Aires,Ecobici,Serttel Brasil,Bike In Baires Consortium.[10],2010,NA,400,4000,21917
5,Argentina,Rosario,Mi Bici Tu Bici[11],NA,NA,2 December 2015,NA,47,480,NA
6,Australia,Melbourne[12],Melbourne Bike Share,PBSC & 8D,Motivate,June 2010,30 November 2019[13],53,676,NA


,City,CITY_ASCII,LAT,LNG,COUNTRY,ISO2,ISO3,ADMIN_NAME,CAPITAL,POPULATION,ID
,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,Tokyo,Tokyo,35.6897,139.6922,Japan,JP,JPN,TÅkyÅ,primary,37977000,1392685764
2,Jakarta,Jakarta,-6.2146,106.8451,Indonesia,ID,IDN,Jakarta,primary,34540000,1360771077
3,Delhi,Delhi,28.6600,77.2300,India,IN,IND,Delhi,admin,29617000,1356872604
4,Mumbai,Mumbai,18.9667,72.8333,India,IN,IND,MahÄrÄshtra,admin,23355000,1356226629
5,Manila,Manila,14.5958,120.9772,Philippines,PH,PHL,Manila,primary,23088000,1608618140
6,Shanghai,Shanghai,31.1667,121.4667,China,CN,CHN,Shanghai,admin,22120000,1156073548


You can use the following dataset list and the `names()` function to get and set each of their column names, and convert them according to our defined naming convention.


In [12]:
dataset_list <- c('raw_bike_sharing_systems.csv', 'raw_seoul_bike_sharing.csv', 'raw_cities_weather_forecast.csv', 'raw_worldcities.csv')

*TODO*: Write a `for` loop to iterate over the above datasets and convert their column names


In [13]:
setwd("D:")
for (dataset_name in dataset_list){
    # Read dataset
    dataset <- read_csv(dataset_name)
    # Standardized its columns:
    convert<-toupper(colnames(dataset))
    colnames(dataset)<- convert
    # Convert all column names to uppercase
   dataset_fixed<-dataset %>% mutate_all(funs(str_replace(.," ","_")))
    print(dataset_fixed)
    # Replace any white space separators by underscores, using the str_replace_all function
    # Save the dataset 
    write.csv(dataset_fixed, dataset_name, row.names=FALSE)
}
                 
                 
                 
                 

Rows: 480 Columns: 10
-- Column specification --------------------------------------------------------
Delimiter: ","
chr (10): COUNTRY, CITY, NAME, SYSTEM, OPERATOR, LAUNCHED, DISCONTINUED, STA...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning message:
"`funs()` was deprecated in dplyr 0.8.0.
Please use a list of either functions or lambdas: 

  # Simple named list: 
  list(mean = mean, median = median)

  # Auto named with `tibble::lst()`: 
  tibble::lst(mean, median)

  # Using lambdas
  list(~ mean(., trim = .2), ~ median(., na.rm = TRUE))
This warning is displayed once every 8 hours.
Call `lifecycle::last_lifecycle_warnings()` to see where this warning was generated."


# A tibble: 480 x 10
   COUNTRY   CITY  NAME  SYSTEM OPERATOR LAUNCHED DISCONTINUED STATIONS BICYCLES
   <chr>     <chr> <chr> <chr>  <chr>    <chr>    <chr>        <chr>    <chr>   
 1 Albania   Tira~ Ecov~ NA     NA       March_2~ NA           8        200     
 2 Argentina Mend~ Metr~ NA     NA       2014     NA           2        40      
 3 Argentina San_~ Bici~ Biciu~ NA       27_Nove~ NA           8        80      
 4 Argentina Buen~ Ecob~ Sertt~ Bike_In~ 2010     NA           400      4000    
 5 Argentina Rosa~ Mi_B~ NA     NA       2_Decem~ NA           47       480     
 6 Australia Melb~ Melb~ PBSC_~ Motivate June_20~ 30_November~ 53       676     
 7 Australia Bris~ City~ 3_Gen~ JCDecaux Septemb~ NA           150      2000    
 8 Australia Melb~ oBike 4_Gen~ NA       July_20~ July_2018    dockless 1250    
 9 Australia Sydn~ oBike 4_Gen~ NA       July_20~ July_2018    dockless 1250    
10 Australia Sydn~ Ofo   4_Gen~ NA       October~ NA           dockless 600     
# ... w

Rows: 8760 Columns: 14
-- Column specification --------------------------------------------------------
Delimiter: ","
chr  (4): DATE, SEASONS, HOLIDAY, FUNCTIONING_DAY
dbl (10): RENTED_BIKE_COUNT, HOUR, TEMPERATURE, HUMIDITY, WIND_SPEED, VISIBI...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


# A tibble: 8,760 x 14
   DATE       RENTED_BIKE_COUNT HOUR  TEMPERATURE HUMIDITY WIND_SPEED VISIBILITY
   <chr>      <chr>             <chr> <chr>       <chr>    <chr>      <chr>     
 1 01/12/2017 254               0     -5.2        37       2.2        2000      
 2 01/12/2017 204               1     -5.5        38       0.8        2000      
 3 01/12/2017 173               2     -6          39       1          2000      
 4 01/12/2017 107               3     -6.2        40       0.9        2000      
 5 01/12/2017 78                4     -6          36       2.3        2000      
 6 01/12/2017 100               5     -6.4        37       1.5        2000      
 7 01/12/2017 181               6     -6.6        35       1.3        2000      
 8 01/12/2017 460               7     -7.4        38       0.9        2000      
 9 01/12/2017 930               8     -7.6        37       1.1        2000      
10 01/12/2017 490               9     -6.5        27       0.5        1928      
# ...

Rows: 160 Columns: 12
-- Column specification --------------------------------------------------------
Delimiter: ","
chr (4): CITY, WEATHER, SEASON, FORECAST_DATETIME
dbl (8): VISIBILITY, TEMP, TEMP_MIN, TEMP_MAX, PRESSURE, HUMIDITY, WIND_SPEE...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


# A tibble: 160 x 12
   CITY  WEATHER VISIBILITY TEMP  TEMP_MIN TEMP_MAX PRESSURE HUMIDITY WIND_SPEED
   <chr> <chr>   <chr>      <chr> <chr>    <chr>    <chr>    <chr>    <chr>     
 1 Seoul Clear   10000      12.32 10.91    12.32    1015     50       2.18      
 2 Seoul Clear   10000      11.48 9.81     11.48    1016     48       1.25      
 3 Seoul Clouds  10000      9.99  8.82     9.99     1015     46       0.94      
 4 Seoul Clouds  10000      7.87  7.87     7.87     1014     46       0.83      
 5 Seoul Clouds  10000      10.09 10.09    10.09    1014     37       1.96      
 6 Seoul Rain    10000      9.74  9.74     9.74     1014     48       3.24      
 7 Seoul Clouds  10000      11.39 11.39    11.39    1012     44       5.65      
 8 Seoul Rain    10000      9.75  9.75     9.75     1012     57       5.63      
 9 Seoul Rain    10000      9.11  9.11     9.11     1013     51       4.91      
10 Seoul Clear   10000      8.33  8.33     8.33     1013     62       4.49      
# ... w

Rows: 26569 Columns: 11
-- Column specification --------------------------------------------------------
Delimiter: ","
chr (7): CITY, CITY_ASCII, COUNTRY, ISO2, ISO3, ADMIN_NAME, CAPITAL
dbl (4): LAT, LNG, POPULATION, ID

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


# A tibble: 26,569 x 11
   CITY        CITY_ASCII  LAT      LNG   COUNTRY ISO2  ISO3  ADMIN_NAME CAPITAL
   <chr>       <chr>       <chr>    <chr> <chr>   <chr> <chr> <chr>      <chr>  
 1 Tokyo       Tokyo       35.6897  139.~ Japan   JP    JPN   Tokyo      primary
 2 Jakarta     Jakarta     -6.2146  106.~ Indone~ ID    IDN   Jakarta    primary
 3 Delhi       Delhi       28.66    77.23 India   IN    IND   Delhi      admin  
 4 Mumbai      Mumbai      18.9667  72.8~ India   IN    IND   Maharasht~ admin  
 5 Manila      Manila      14.5958  120.~ Philip~ PH    PHL   Manila     primary
 6 Shanghai    Shanghai    31.1667  121.~ China   CN    CHN   Shanghai   admin  
 7 S<U+FFFD>o_Paulo   Sao_Paulo   -23.5504 -46.~ Brazil  BR    BRA   S<U+FFFD>o_Paulo  admin  
 8 Seoul       Seoul       37.5833  127   Korea,~ KR    KOR   Seoul      primary
 9 Mexico_City Mexico_City 19.4333  -99.~ Mexico  MX    MEX   Ciudad_de~ primary
10 Guangzhou   Guangzhou   23.1288  113.~ China   CN    CHN   Guangdong

*TODO*: Read the resulting datasets back and check whether their column names follow the naming convention


In [14]:
for (dataset_name in dataset_list){
    # Print a summary for each data set to check whether the column names were correctly converted
    datasets<-read.csv(dataset_name)
    print(summary(datasets))
}

   COUNTRY              CITY               NAME              SYSTEM         
 Length:480         Length:480         Length:480         Length:480        
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  
   OPERATOR           LAUNCHED         DISCONTINUED         STATIONS        
 Length:480         Length:480         Length:480         Length:480        
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  
   BICYCLES         DAILY_RIDERSHIP   
 Length:480         Length:480        
 Class :character   Class :character  
 Mode  :character   Mode  :character  
     DATE           RENTED_BIKE_COUNT      HOUR        TEMPERATURE    
 Length:8760        Min.   :   2.0    Min.   : 0.00   Min.   :-17.80  
 Class :character   1st Qu.: 214.0    1st Qu.: 5.75   1st Qu.:  3.40  
 Mode  :charact

## Process the web-scraped bike sharing system dataset


By now we have standardized all column names. Next, we will focus on cleaning up the values in the web-scraped bike sharing systems dataset.


In [15]:
# First load the dataset
bike_sharing_df <- read_csv("raw_bike_sharing_systems.csv")

Rows: 480 Columns: 10
-- Column specification --------------------------------------------------------
Delimiter: ","
chr (10): COUNTRY, CITY, NAME, SYSTEM, OPERATOR, LAUNCHED, DISCONTINUED, STA...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [16]:
# Print its head
head(bike_sharing_df)

COUNTRY,CITY,NAME,SYSTEM,OPERATOR,LAUNCHED,DISCONTINUED,STATIONS,BICYCLES,DAILY_RIDERSHIP
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Albania,Tirana,Ecovolis,NA,NA,March_2011,NA,8,200,NA
Argentina,Mendoza,Metrobici,NA,NA,2014,NA,2,40,NA
Argentina,"San_Lorenzo,_Santa Fe",Biciudad,Biciudad,NA,27_November_2016,NA,8,80,NA
Argentina,Buenos_Aires,Ecobici,Serttel_Brasil,Bike_In_Baires Consortium.[10],2010,NA,400,4000,21917
Argentina,Rosario,Mi_Bici_Tu Bici[11],NA,NA,2_December_2015,NA,47,480,NA
Australia,Melbourne[12],Melbourne_Bike_Share,PBSC_&_8D,Motivate,June_2010,30_November_2019[13],53,676,NA


Even from the first few rows, you can see there is plenty of undesireable embedded textual content, such as the reference link included in `Melbourne[12]`.


In this project, let's only focus on processing the following revelant columns (feel free to process the other columns for more practice):

*   `COUNTRY`: Country name
*   `CITY`: City name
*   `SYSTEM`: Bike-sharing system name
*   `BICYCLES`: Total number of bikes in the system


In [17]:
# Select the four columns
sub_bike_sharing_df <- bike_sharing_df %>% select(COUNTRY, CITY, SYSTEM, BICYCLES)

Let's see the types of the selected columns


In [18]:
sub_bike_sharing_df %>% 
    summarize_all(class) %>%
    gather(variable, class)

variable,class
<chr>,<chr>
COUNTRY,character
CITY,character
SYSTEM,character
BICYCLES,character


They are all interpreted as character columns, but we expect the `BICYCLES` column to be of numeric type. Let's see why it wasn't loaded as a numeric column - possibly some entries contain characters. Let's create a simple function called `find_character` to check that.


In [19]:
# grepl searches a string for non-digital characters, and returns TRUE or FALSE
# if it finds any non-digital characters, then the bicyle column is not purely numeric
find_character <- function(strings) grepl("[^0-9]", strings)
find_character

function(strings) grepl("[^0-9]", strings)

Let's try to find any elements in the `Bicycles` column containing non-numeric characters.


In [24]:
sub_bike_sharing_df %>% 
    select(BICYCLES) %>% 
    filter(find_character(BICYCLES)) %>%
    slice(0:10)

BICYCLES
<chr>
4115[22]
310[59]
500[72]
[75]
180[76]
600[77]
[78]
initially_800_(later 2500)
100_(220)


As you can see, many rows have non-numeric characters, such as `32 (including 6 rollers) [162]` and `1000[253]`. This is actually very common for a table scraped from Wiki when no input validation is enforced.

Later, you will use regular expressions to clean them up.


Next, let's take a look at the other columns, namely `COUNTRY`, `CITY`, and `SYSTEM`, to see if they contain any undesired reference links, such as in `Melbourne[12]`.


In [49]:
# Define a 'reference link' character class, 
# `[A-z0-9]` means at least one character 
# `\\[` and `\\]` means the character is wrapped by [], such as for [12] or [abc]
ref_pattern <- "\\[[A-z0-9]+\\]"
find_reference_pattern <- function(strings) grepl(ref_pattern, strings)

In [26]:
# Check whether the COUNTRY column has any reference links
sub_bike_sharing_df %>% 
    select(COUNTRY) %>% 
    filter(find_reference_pattern(COUNTRY)) %>%
    slice(0:10)

COUNTRY
<chr>


Ok, looks like the `COUNTRY` column is clean. Let's check the `CITY` column.


In [27]:
# Check whether the CITY column has any reference links
sub_bike_sharing_df %>% 
    select(CITY) %>% 
    filter(find_reference_pattern(CITY)) %>%
    slice(0:10)

CITY
<chr>
Melbourne[12]
Brisbane[14][15]
Lower_Austria[18]
Namur[19]
Brussels[21]
Salvador[23]
Belo_Horizonte[24]
Jo<U+FFFD>o_Pessoa[25]
(Pedro_de)_Toledo[26]


Hmm, looks like the `CITY` column has some reference links to be removed. Next, let's check the `SYSTEM` column.


In [28]:
# Check whether the System column has any reference links
sub_bike_sharing_df %>% 
    select(SYSTEM) %>% 
    filter(find_reference_pattern(SYSTEM)) %>%
    slice(0:10)

SYSTEM
<chr>
EasyBike[58]
4_Gen.[61]
3_Gen._SmooveKey[113]
3_Gen._Smoove[141][142][143][139]
3_Gen._Smoove[179]
3_Gen._Smoove[181]
3_Gen._Smoove[183]


So the `SYSTEM` column also has some reference links.


After some preliminary investigations, we identified that the `CITY` and `SYSTEM` columns have some undesired reference links, and the `BICYCLES` column has both reference links and some
textual annotations.

Next, you need to use regular expressions to clean up the unexpected reference links and text annotations in numeric values.


# TASK: Remove undesired reference links using regular expressions


*TODO:* Write a custom function using `stringr::str_replace_all` to replace all reference links with an empty character for columns `CITY` and `SYSTEM`


In [56]:
# remove reference link
remove_ref <- function(strings) {
    #reference_pattern <- "Define a pattern matching a reference link such as [1]"
    reference_pattern <- "\\[[\\w]+\\]"
    # Replace all matched substrings with a white space using str_replace_all()
   result<-str_replace_all(strings,reference_pattern," ")
    # Trim the reslt if you want
     return(result)
}

*TODO:* Use the `dplyr::mutate()` function to apply the `remove_ref` function to the `CITY` and `SYSTEM` columns


In [63]:
result<-sub_bike_sharing_df %>% mutate(CITY=remove_ref(CITY),SYSTEM=remove_ref(SYSTEM),BICYCLES=remove_ref(BICYCLES))


*TODO:* Use the following code to check whether all reference links are removed:


In [64]:
result %>% 
    select(CITY, SYSTEM, BICYCLES) %>% 
    filter(find_reference_pattern(CITY) | find_reference_pattern(SYSTEM) | find_reference_pattern(BICYCLES))

CITY,SYSTEM,BICYCLES
<chr>,<chr>,<chr>


# TASK: Extract the numeric value using regular expressions


*TODO:* Write a custom function using `stringr::str_extract` to extract the first digital substring match and convert it into numeric type For example, extract the value '32' from `32 (including 6 rollers) [162]`.


In [72]:
# Extract the first number
extract_num <- function(columns){
    # Define a digital pattern
    digitals_pattern <- "[0-9]+"
    # Find the first match using str_extract
    extracted <-str_extract(columns,digitals_pattern)
    # Convert the result to numeric using the as.numeric() function
    extracted_value<-as.numeric(extracted)
    return(extracted_value)
}

*TODO:* Use the `dplyr::mutate()` function to apply `extract_num` on the `BICYCLES` column


In [73]:
# Use the mutate() function on the BICYCLES column
result2<- result %>% mutate(BICYCLES=extract_num(BICYCLES))

*TODO:* Use the summary function to check the descriptive statistics of the numeric `BICYCLES` column


In [74]:
summary(result2$BICYCLES)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
      5     100     350    2022    1400   78000      78 

*TODO:* Write the cleaned bike-sharing systems dataset into a csv file called `bike_sharing_systems.csv`


In [81]:
# Write dataset to `bike_sharing_systems.csv`
write.csv(result2,"D:bike_sharing_systems.csv",row.names=FALSE)
read.csv("bike_sharing_systems.csv")

COUNTRY,CITY,SYSTEM,BICYCLES
<chr>,<chr>,<chr>,<int>
Albania,Tirana,NA,200
Argentina,Mendoza,NA,40
Argentina,"San_Lorenzo,_Santa Fe",Biciudad,80
Argentina,Buenos_Aires,Serttel_Brasil,4000
Argentina,Rosario,NA,480
Australia,Melbourne,PBSC_&_8D,676
Australia,Brisbane,3_Gen._Cyclocity,2000
Australia,Melbourne,4_Gen._oBike,1250
Australia,Sydney,4_Gen._oBike,1250


# References:


If you need to refresh your memory about regular expressions, please refer to this good Regular Expression cheat sheet:

<a href="https://www.rstudio.com/wp-content/uploads/2016/09/RegExCheatsheet.pdf?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkRP0321ENSkillsNetwork25371262-2022-01-01" target="_blank">Basic Regular Expressions in R</a>


# Next Steps


Great! Now you have cleaned up the bike-sharing system dataset using regular expressions. Next, you will use other `tidyverse` functions to perform data wrangling on the bike-sharing demand dataset.


## Authors

<a href="https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkRP0321ENSkillsNetwork25371262-2022-01-01" target="_blank">Yan Luo</a>


### Other Contributors

Jeff Grossman


## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-04-08        | 1.0     | Yan        | Initial version created |
|                   |         |            |                         |
|                   |         |            |                         |

## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
